# Advent of Code 2024, Day 12

In [1412]:
with open('input.txt', 'r') as f:
    puzzle_input = f.read().strip()

In [1413]:
puzzle_input = """OOOOO
OXOXO
OOOOO
OXOXO
OOOOO"""

In [1414]:
import networkx as nx
import numpy as np

## [First Puzzle:](https://adventofcode.com/2024/day/12)

In [1415]:
matrix = np.array([list(row) for row in puzzle_input.split("\n")])

G = nx.grid_2d_graph(*matrix.shape)

for (x1, y1), (x2, y2) in G.edges:
    if matrix[x1, y1] == matrix[x2, y2]:
        G.add_edge((x1, y1), (x2, y2), edge_type="region")
    else:
        G.add_edge((x1, y1), (x2, y2), edge_type="boundary")

region_edges = [(u, v) for u, v, d in G.edges(data=True) if d["edge_type"] == "region"]
region_subgraph = nx.Graph()
region_subgraph.add_edges_from(region_edges)

for node in G.nodes:
    if node not in region_subgraph:
        region_subgraph.add_node(node)

regions = []
for component in nx.connected_components(region_subgraph):
    representative_node = next(iter(component))
    region_value = matrix[representative_node]
    regions.append((region_value, list(component)))

In [1416]:
region_area_perimeter = [{'region': region, 
  'area': len(region[1]),
  'perimeter': sum(1 for node in region[1] for neighbor in G.neighbors(node) if G[node][neighbor].get("edge_type") == "boundary")
} for region in regions]

In [1417]:
# Add matrix bounds to perimeter total
for rap in region_area_perimeter:
    for node in rap['region'][1]:
        if node[0] == 0:
            rap['perimeter'] += 1
        if node[0] == matrix.shape[0] - 1:
            rap['perimeter'] += 1
        if node[1] == 0:
            rap['perimeter'] += 1
        if node[1] == matrix.shape[1] - 1:
            rap['perimeter'] += 1

In [1418]:
region_prices = [rap['area'] * rap['perimeter'] for rap in region_area_perimeter]

In [1419]:
sum(region_prices)

772

## [Second Puzzle:](https://adventofcode.com/2024/day/12/#part2)

In [1420]:
num_matrix = np.zeros_like(matrix, dtype=int)
for i in range(len(regions)):
    for node in regions[i][1]:
        num_matrix[node] = i + 1
        
# Gotta upsample it since a pixel isn't actually a square and counting inner sides gets weird because of that
num_matrix = np.repeat(np.repeat(num_matrix, 4, axis=0), 4, axis=1)

# Add padding to make sure the contours are closed
num_matrix = np.pad(num_matrix, pad_width=1, mode='constant', constant_values=0)

In [1421]:
matrix

array([['O', 'O', 'O', 'O', 'O'],
       ['O', 'X', 'O', 'X', 'O'],
       ['O', 'O', 'O', 'O', 'O'],
       ['O', 'X', 'O', 'X', 'O'],
       ['O', 'O', 'O', 'O', 'O']], dtype='<U1')

In [1422]:
num_matrix

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],
       [0, 1, 1, 1, 1, 4,

In [1423]:
from skimage.measure import find_contours
from shapely.geometry import Polygon

contours = [np.concatenate(find_contours(num_matrix == i, level=0)) for i in range(1, len(regions) + 1)]

polygons = [set(Polygon(contour).simplify(1, preserve_topology=True).exterior.coords[:-1]) for contour in contours]

In [1424]:
contours[2]

array([[ 9., 16.],
       [ 9., 15.],
       [ 9., 14.],
       [ 9., 13.],
       [ 8., 12.],
       [ 7., 12.],
       [ 6., 12.],
       [ 5., 12.],
       [ 4., 13.],
       [ 4., 14.],
       [ 4., 15.],
       [ 4., 16.],
       [ 5., 17.],
       [ 6., 17.],
       [ 7., 17.],
       [ 8., 17.],
       [ 9., 16.]])

In [1425]:
polygons[2]

{(4.0, 13.0), (5.0, 17.0), (8.0, 12.0), (9.0, 16.0)}

In [1426]:
region_prices = [region_area_perimeter[i]['area'] * len(polygons[i]) for i in range(len(regions))]

In [1427]:
sum(region_prices)

688